In [23]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
import pandas as pd
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os
import pickle
from gensim.models import CoherenceModel
import json
import pandas as pd
import spacy        #per la lemmizzazione delle parole
from wordcloud import WordCloud
import string
from stop_words import get_stop_words
from nltk.corpus import stopwords
import json
import glob
import re
import json
import matplotlib.pyplot as plt
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wissel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
def read_setting_data():
           

           
            base_path=os.path.join("model_settings","model_config.json")
            # Open the JSON file and load its contents
            with open(base_path, 'r') as file:
                data = json.load(file)
                clusters=int(data["number_clusers"])
                return clusters

In [25]:
model="model_"+str(read_setting_data())
model_path=model+"/model.lda"
dic_path=model+"/id2token.mm"
lda_model = gensim.models.LdaModel.load(model_path)
dictionary = corpora.Dictionary.load(dic_path)
               

In [26]:
    #pipeline di pulizia
def remove_stops(text,stops):
        #rimozione data
        pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b'
        text= re.sub(pattern, '', text)
        text=text.replace("\n"," ")
        text=text.replace("-"," ")
        text=text.strip()
        text=text.lower()
        words=text.split()
        
        
        
        #rimozione stopwords
        final=[]
        for word in words:
            if word not in stops:
                if len(word)<16 and len(word)>1:
                    final.append(word)
        final=" ".join(final)

        #punti
        final=final.translate(str.maketrans("","",string.punctuation))
        #rimozione numeri
        final="".join([i for i in final if not i.isdigit()])

        #eliminazione doppi " "
        while "  " in final:
            final=final.replace("  "," ")
        
        return (final)

def remove_custom_punct(text):
        translator = str.maketrans('', '', ''.join(string.punctuation))
        return text.translate(translator)

def remove_doubleCharWords(line):
        words=line.split()
        final=[]
        for i in words:
            if len(i)>2:
                words=remove_custom_punct(i)
                final.append(words)
            
                
        final=" ".join(final)

        return final
        
def lemming_text( text):
        # Load the Italian language model
        nlp = spacy.load("it_core_news_sm")
        # Process the text with spaCy
        doc = nlp(text)
        # Lemmatize each token in the text
        lemmas = [token.lemma_ for token in doc]
        # Print the lemmas
        text=" ".join(lemmas)
        return text
    
            
def find_none_spaced_words(text):
       
       
            
        text_with_space = re.sub(r"(\w)([A-Z])", r"\1 \2", text)

        
        return text_with_space

def delete_double_spaces(text):
        text_without_double_spaces = re.sub(r"\s+", " ", text)
        return text_without_double_spaces

In [34]:
def parse_query(i):
    
            new_text=[]
            stops=stopwords.words("italian")
            stops.extend(stopwords.words("english"))
        

            if type(i)==type("s"):
                #rimozione di tutte le parole che non hanno lunghezza maggiore di 2
                parsed_text=find_none_spaced_words(i)              #andiamo a dividere eventuali parole distinte, concatenate (es. HelloWord->Hello World)
                parsed_text=remove_stops(parsed_text,stops)                  #rimozione stopwords
                parsed_text=remove_doubleCharWords(parsed_text)    #rimozione parole e simboli superflui
                parsed_text=delete_double_spaces(parsed_text)
                parsed_text=lemming_text(parsed_text)              #lemming delle parole
                
            else:
                parsed_text=""
                
            return parsed_text

        

In [36]:
 # Preprocess the new text
new_text = "complessità asistotica "
new_text = parse_query(new_text).split()
new_text_bow = dictionary.doc2bow(new_text)
    
# Get the topics related to the new text
topics = lda_model.get_document_topics(new_text_bow)

 # Print the topics
for topic in topics:
                print("Topic {}: {:.2%}".format(topic[0], topic[1]))
